In [1]:
import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.5.0"  # Ajusta si usas otra versión


In [2]:
import pandas as pd

from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# Habilitar conversión entre pandas y R
pandas2ri.activate()

In [3]:
from rpy2.robjects.packages import importr

# Cargar el paquete base de R
base = importr('base')

In [4]:
# Verificar conexión con R e intentar cargar un paquete
utils = importr('utils')  # Librería para gestionar paquetes en R

try:
    depmixS4 = importr('depmixS4')  # Intentar cargar depmixS4
    print("El paquete 'depmixS4' se cargó correctamente.")
except:
    print("El paquete 'depmixS4' no está instalado. Procediendo a instalar...")
    utils.install_packages('depmixS4')
    depmixS4 = importr('depmixS4')
    print("El paquete 'depmixS4' se instaló y cargó correctamente.")


El paquete 'depmixS4' se cargó correctamente.


In [6]:
# 1. Cargar la base de datos original
covid_data = pd.read_csv("../data/covid_data.csv")

In [7]:
covid_data.head()

,index,IM_interno,IM_externo,inc_pc_comuna,TasaCasosAcumulados,Personas_hogar_promedio,inmigrantes,total_hog_hacinados,indic_precariedad_vivienda,Comuna,...,paso,paso_etiqueta,Fecha inicio cuarentena,Fecha término cuarentena,Casos_Diarios,Casos_Diarios_suavizado,aceleracion,Casos_Diarios_mediamovil,aceleracion_mediamovil,TasaCasosAcumulados_mediamovil
0,7,1.735568,8.083123,267197.56,5.620756,3.145581,3606,68120,3.395792,Cerrillos,...,0,Sin Medida,2020-05-05,2020-08-31,2.127858,0.802965,1.097386,2.236832,0.573547,8.511431
1,8,1.732670,8.235757,267197.56,8.993210,3.145581,3606,68120,3.395792,Cerrillos,...,0,Sin Medida,2020-05-05,2020-08-31,3.171712,0.963558,1.204448,2.753024,0.520016,11.241513
2,9,1.663978,8.296887,267197.56,12.365664,3.145581,3606,68120,3.395792,Cerrillos,...,0,Sin Medida,2020-05-05,2020-08-31,3.894381,1.124151,0.776200,3.240538,0.409130,14.453373
3,10,1.463511,8.628372,267197.56,17.986420,3.145581,3606,68120,3.395792,Cerrillos,...,0,Sin Medida,2020-05-05,2020-08-31,4.215567,1.927116,-0.133828,3.642021,0.347952,17.825827
4,11,0.921666,5.521175,267197.56,23.607177,3.145581,3606,68120,3.395792,Cerrillos,...,0,Sin Medida,2020-05-05,2020-08-31,4.054974,2.730082,-0.508545,3.951736,0.302068,21.680060
